In [1]:
import json
import re

file_path = "cs.json"
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

In [3]:
products = []
b = []
failed = []
dups = 0
base_url = 'https://coldstorage.com.sg'

In [4]:
#just for checking
offdisp = []
size = []
price_bef = []
price_aft = []
total = 0

In [5]:
def get_offer_qty(item):
    
    price_bef.append(item['price'])
    price = float(re.findall(r'\d+(?:\.\d+)?', item['price'])[0])
    
    if item['offer_display']:
        #need to do some string splitting here
        if '% off' in item['offer_display']:
            perc = int(item['offer_display'].strip(' % off'))
            price = round(price/max(perc, 0.000001)*100, 2)
            price_aft.append(price)
            return None, item['price'],item['offer_display'], price #change later
        else:
            _, qty, _, i = item['offer_display'].split(' ')
            return qty, i ,item['offer_display'], price
    else:
        return None, None, None, price

for cat, items in data.items():
    for item in items:
        total+=1
        product = {}
        present = False
        name = item['name']
        
        #check for duplicates
        if name in b:
            #PRODUCT already within dict, add additional category if needed
            present = True
            dups+=1
        else:
            b.append(name)
            
        
        if present:
            continue
            
        try:
            product['product_url'] = base_url + item['url_to_product']
            product['market_id'] = 2
            product['barcodes'] = None
            product['brand'] = item['brand']
            product['availability'] = True #by default
            product['name'] = item['name']
            
            try:
                product['image'] = item['image_url']
            except:
                product['image'] = None
                
            try:
                size.append(item['size']) #remove later
                product['quantity'] = item['size'].strip('Size: ') #need edit
                 
            except:
                product['quantity'] = None
                #print('size problem')
                
            offdisp.append(item['offer_display'])
            product['offer_qty'],product['offer_price'],product['offer_desc'],product['price'] = get_offer_qty(item)
       
            products.append(product)
        except Exception as e:
            failed.append([item, e]) 

In [6]:
len(failed)

0

In [7]:
len(products)

7259

In [8]:
dups

1002

In [ ]:
b

In [9]:
products[0]

{'product_url': 'https://coldstorage.com.sg/sliced-emmental-5030672',
 'market_id': 2,
 'barcodes': None,
 'brand': 'TRADITION',
 'availability': True,
 'name': 'Emmental Slices 125g',
 'image': 'https://coldstorage.com.sg/vcpimg/thumb/5030672-1.jpg',
 'quantity': '125G',
 'offer_qty': None,
 'offer_price': None,
 'offer_desc': None,
 'price': 4.1}

In [10]:
import json

rand = {}
rand['all'] = products

file_path = "cs_cleaned.json"
with open(file_path, 'w') as json_file:
    json.dump(rand, json_file)